In [1]:
## Import things:
import pandas as pd 
import os
import shutil
import datetime 

## Constants 
study = ["actual"]
og_wd = os.getcwd()
data_file_path  =  str(og_wd)
data_file_path = data_file_path.replace(os.sep, '/')
models_path = data_file_path + '/Models/'
schedules_path = data_file_path + '/Schedule Files/'
original_osm = "bldg#######.osm"
original_weather = "weather.epw"
original_schedule_name = "schedules.csv"
now = "actual" #datetime.datetime.now()


In [25]:
# Modfiy weather file name here:
actual_weather_files = os.listdir(data_file_path + "/Weather")

for each_file in actual_weather_files:
    shutil.copyfile(data_file_path + "/Weather/" + each_file,data_file_path + "/" + each_file)

In [26]:
# Identify all the models
models = []
# Iterate directory
for model in os.listdir(models_path):
    # check only text files
    if model.endswith('.osm'):
        models.append(model)

In [27]:
#Create new output folders for each scenario 
for each_study in study:
    new_path = data_file_path + "/Outputs/" + each_study
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    for each_year in actual_weather_files:
        newest_path = new_path + "/" + each_year[:-14]
        if not os.path.exists(newest_path):
            os.makedirs(newest_path)
        for each_model in models:
            model_path = newest_path + "/" + each_model[:-4]
            if not os.path.exists(model_path):
                os.makedirs(model_path)

In [31]:
# Start Simulating
for each_study in study:
    if each_study == 'actual':
        for new_weather in actual_weather_files:
            year = new_weather[:-14]
            for each_model in models:
                new_path = data_file_path + "/Outputs/actual/" + year + "/" + each_model[:-4]
                this_schedules_path = data_file_path + "/Outputs/baseline/" + each_model[:-4]  + "/"+ each_model[:-4] + '_baseline.csv'

                # Save schedule in main folder as 'schedules.csv' (IF YOU DO NOT DO THIS, IT WILL NOT RUN)
                shutil.copyfile(this_schedules_path,original_schedule_name)

                # this_schedules = each_model[:-4] + '_baseline.csv'
                # this_schedules_path = new_path + "/" + this_schedules_name
                # copy the baseline template and paste
                shutil.copyfile(data_file_path + "/run_all_models_baseline.osw",data_file_path + "/run_this_model.osw")
                # open the file as a readable file
                with open(data_file_path + "/run_this_model.osw",'r') as file:
                    data = file.read()

                    new_osm = data_file_path  + "/Models/" + each_model

                    #replace with new osm
                    data = data.replace(original_osm, new_osm)

                    # replace with new weather file (baseline)
                    data = data.replace(original_weather,new_weather)

                with open(data_file_path + "/run_this_model.osw",'w') as file:
                    file.write(data)

                # run 
                file_to_run = data_file_path + "/run_this_model.sh"
                os.system(file_to_run)

                #save actual osm in output folder
                destination = new_path + "/" + each_model[:-4] + "_actual.osm"
                shutil.copyfile(new_osm,destination)

                # change output filename and save in data_folder
                source = data_file_path + "/run/eplusout.sql"
                destination = new_path + "/eplusout.sql"
                shutil.copyfile(source,destination)

                # change output filename and save in data_folder
                source = data_file_path + "/run/eplusout.eso"
                destination = new_path + "/eplusout.eso"
                shutil.copyfile(source,destination)

                # change output filename and save in data_folder
                source = data_file_path + "/run/eplusout.err"
                destination = new_path + "/eplusout.err"
                shutil.copyfile(source,destination)